In [ ]:
pip install beautifulsoup4


In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [ ]:
requests.get('https://www.ambitionbox.com/list-of-companies?campaign=desktop_nav&page=3')

In [ ]:
import requests

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.162 Safari/537.36'
}

response = requests.get(
    'https://www.ambitionbox.com/list-of-companies?campaign=desktop_nav&page=1',
    headers=headers
)

html_text = response.text


In [ ]:
html_text

In [ ]:
pip install lxml


In [ ]:
soup = BeautifulSoup(html_text, 'html.parser')


In [ ]:
print(soup.prettify())

In [ ]:
soup.find_all('h1')[0].text

In [ ]:
soup.find_all('h2')

In [ ]:
len(soup.find_all('h2'))

In [ ]:
for i in soup.find_all('h2'):
    print(i.text)

In [ ]:
for i in soup.find_all('h2'):
    print(i.text.strip())

In [ ]:
for i in soup.find_all('p'):
    print(i.text.strip())

In [ ]:
soup.find_all('p', class_='rating')

In [ ]:
soup.find_all('a', class_='review_count')

In [ ]:
for i in soup.find_all('p', class_='rating'):
    print(i.text)

In [ ]:
company = soup.find_all('div', class_='companyCardWrapper')


In [ ]:
len(company)

In [ ]:
name=[]

for i in company:
    print(i.find('h2').text.strip())

In [ ]:
rating=[]

for i in company:
    name.append(i.find('h2').text.strip())
    # rating.append(i.find_all('div', class_='rating_text').text.strip())
    ratings = i.find_all('div', class_='rating_text')
    rating.append([r.text.strip() for r in ratings])


In [ ]:
ratings = i.find_all('div', class_='rating_text')
rating.append([r.text.strip() for r in ratings])


In [ ]:
name


In [ ]:
rating

In [ ]:
reviews = []

for i in company:
    review_links = i.find_all('a', class_='companyCardWrapper__ActionWrapper')
    review_count = 'N/A'
    for link in review_links:
        title_span = link.find('span', class_='companyCardWrapper__ActionTitle')
        if title_span and title_span.text.strip() == 'Reviews':
            count_span = link.find('span', class_='companyCardWrapper__ActionCount')
            if count_span:
                review_count = count_span.text.strip()
            break
    reviews.append(review_count)


In [ ]:
reviews

In [ ]:
c_type = []
hq = []
old = []
employees = []

for i in company:
    # Company Type (e.g., IT Services & Consulting)
    type_tag = i.find('span', class_='companyCardWrapper__interLinking')
    c_type.append(type_tag.text.strip() if type_tag else 'N/A')

    # Headquarters, Founded Year, Employee Count — assuming these are in <p> tags with identifiable classes
    # You’ll need to inspect the actual HTML to confirm these selectors
    hq_tag = i.find('p', class_='headquarters')  # hypothetical class
    hq.append(hq_tag.text.strip() if hq_tag else 'N/A')

    old_tag = i.find('p', class_='founded')  # hypothetical class
    old.append(old_tag.text.strip() if old_tag else 'N/A')

    emp_tag = i.find('p', class_='employee-count')  # hypothetical class
    employees.append(emp_tag.text.strip() if emp_tag else 'N/A')


In [ ]:
name
rating
reviews
c_type
hq
old
employees

In [ ]:
print("name:", len(name))
print("rating:", len(rating))
print("reviews:", len(reviews))
print("company_type:", len(c_type))
print("headquarters:", len(hq))
print("founded:", len(old))
print("employees:", len(employees))


In [ ]:
import pandas as pd

d = {
    'name': name,
    'rating': rating,
    'reviews': reviews,
    'company_type': c_type,
    'headquarters': hq,
    'founded': old,
    'employees': employees
}

df = pd.DataFrame(d)


In [ ]:
df

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time

all_data = []

for j in range(1, 1001):
    print(f"Scraping page {j}...")
    url = f'https://www.ambitionbox.com/list-of-companies?page={j}'
    webpage = requests.get(url).text
    soup = BeautifulSoup(webpage, 'html.parser')  # fallback parser
    company = soup.find_all('div', class_='company-content-wrapper')

    name = []
    rating = []
    reviews = []
    ctype = []
    hq = []
    how_old = []
    no_of_employee = []

    for i in company:
        try:
            name.append(i.find('h2').text.strip())
        except:
            name.append(np.nan)

        try:
            rating.append(i.find('p', class_='rating').text.strip())
        except:
            rating.append(np.nan)

        try:
            reviews.append(i.find('a', class_='review-count').text.strip())
        except:
            reviews.append(np.nan)

        info_entities = i.find_all('p', class_='infoEntity')
        try:
            ctype.append(info_entities[0].text.strip())
        except:
            ctype.append(np.nan)
        try:
            hq.append(info_entities[1].text.strip())
        except:
            hq.append(np.nan)
        try:
            how_old.append(info_entities[2].text.strip())
        except:
            how_old.append(np.nan)
        try:
            no_of_employee.append(info_entities[3].text.strip())
        except:
            no_of_employee.append(np.nan)

    df = pd.DataFrame({
        'name': name,
        'rating': rating,
        'reviews': reviews,
        'company_type': ctype,
        'Head_Quarters': hq,
        'Company_Age': how_old,
        'No_of_Employee': no_of_employee
    })

    all_data.append(df)
    time.sleep(1)  # polite delay to avoid hammering the server

# Combine all pages into one DataFrame
final = pd.concat(all_data, ignore_index=True)


In [ ]:
final.drop_duplicates(inplace=True)
final.dropna(subset=['name', 'rating'], inplace=True)


In [ ]:
pip install seaborn matplotlib

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.histplot(final['rating'].dropna(), bins=20)
plt.title('Distribution of Company Ratings')
plt.show()


In [ ]:
final.to_csv('ambitionbox_companies_cleaned.csv', index=False)
